In [1]:
## import dependencies

from mixpanel_api import Mixpanel
import pandas as pd
from pandas.io.json import json_normalize
import requests
import json
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
## import data from MixPanel

if __name__ == '__main__':
  credentials = {
    'API_secret': '..',
    'token': '',
  }

  m = Mixpanel(credentials['API_secret'])
# selector = ''  
  selector = '(("US" in properties["$country_code"]) and (defined (properties["$country_code"])))'
  parameters = {'selector': selector}

m.export_people('people_export.txt',parameters,timezone_offset=-8)

In [3]:
## import txt file as json

with open('people_export.txt', 'r') as handle:
    parsed = json.load(handle)

## printing the json file, if needed    
#print(json.dumps(parsed, indent=4, sort_keys=True))

In [4]:
## convert json to dataframe

people_df = pd.DataFrame.from_dict(json_normalize(parsed), orient='columns')
#people_df.head(10)

In [5]:
len(people_df)

294710

In [6]:
selector = '(("Brooklyn" in properties["$city"]) and (defined (properties["$city"])))'
parameters = {'selector': selector}
m.export_events('event_export.txt',{'from_date':'2019-06-01','to_date':'2019-06-02'},parameters)

Dumping json to event_export.txt


In [7]:
with open('event_export.txt', 'r') as handle:
    events = json.load(handle)
## printing the json file, if needed    
#print(json.dumps(parsed, indent=4, sort_keys=True))

In [8]:
events_df = pd.DataFrame.from_dict(json_normalize(events), orient='columns')
#events_df.head(10)

In [9]:
len(events_df)

629396

In [10]:
events_describe = events_df.groupby('event').count()
events_describe = events_describe.loc[:,'properties.time'].reset_index()

In [11]:
#events_describe